In [ ]:
import pandas as pd
from os import listdir

In [ ]:
cab_types = {
    'fhv':0,
    'green':1,
    'yellow':2,
}
filenames = [x for x in listdir('data') if not x.startswith('fhv_')]
paths = ['data/' + x for x in filenames]
cab_types_per_path = [cab_types[x.split('_')[0]] for x in filenames]
filenames

In [ ]:
dfs = [pd.read_csv(
    f, 
    nrows=100,
    # usecols=[
    #     'passenger_count',
    #     'pickup_datetime',
    #     'trip_distance',
    #     'total_amount',
    # ],
    dtype={
        'passenger_count': int,
        'pickup_datetime': str,
        'trip_distance':float,
        'total_amount':float,
    }
) for f in paths]

In [ ]:
for i in range(len(dfs)):
    dfs[i]['cab_type'] = cab_types_per_path[i]

In [ ]:
trips = pd.concat(dfs)

In [ ]:
trips.shape

In [ ]:
pd.set_option('display.max_columns', None)
trips.head()

### Query 1

It's crucial to run `groupby` queries on integers. When working on a strings, processing a small sample took 1m5s, with integers - only 3s.

In [ ]:
selected_df = trips[['cab_type']]
grouped_df = selected_df.groupby('cab_type')
final_df = grouped_df.size().reset_index(name='counts')
final_df

### Query 2

In [ ]:
selected_df = trips[['passenger_count', 'total_amount']]
grouped_df = selected_df.groupby('passenger_count')
final_df = grouped_df.mean().reset_index()
final_df

### Query 3

In [ ]:
# TODO: Debug the rewriting warning.
# TODO: understand why 2018 files contain 2019 year.

pickup_filed_name = 'lpep_pickup_datetime' # not the 'pickup_datetime', 'lpep_pickup_datetime'
selected_df = trips[['passenger_count', pickup_filed_name]]
selected_df['year'] = pd.to_datetime(selected_df.pop(pickup_filed_name), format='%Y-%m-%d %H:%M:%S').dt.year
grouped_df = selected_df.groupby(['passenger_count', 'year'])
final_df = grouped_df.size().reset_index(name='counts')
final_df

### Query 4

In [ ]:
selected_df = trips[['passenger_count', pickup_filed_name, 'trip_distance']]
selected_df['trip_distance'] = selected_df['trip_distance'].round().astype(int)
selected_df['year'] = pd.to_datetime(selected_df.pop(pickup_filed_name), format='%Y-%m-%d %H:%M:%S').dt.year
grouped_df = selected_df.groupby(['passenger_count', 'year', 'trip_distance'])
final_df = grouped_df.size().reset_index(name='counts').copy()
final_df = final_df.sort_values(['year', 'counts'], ascending=[True, False]) 
final_df